<a href="https://colab.research.google.com/github/junkyuhufs/stats/blob/main/WoS_text_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##[Anonymous]로 reference삭제. CSV파일에서

In [1]:
# STEP 1: 파일 업로드
from google.colab import files
import pandas as pd

uploaded = files.upload()  # 사용자로부터 CSV 업로드 받기
filename = next(iter(uploaded))  # 첫 번째 업로드 파일 이름

# STEP 2: CSV 파일 읽기 (인코딩 자동 감지)
encodings = ['utf-8', 'cp949', 'ISO-8859-1']
for enc in encodings:
    try:
        df = pd.read_csv(filename, encoding=enc)
        print(f"✅ Encoding detected: {enc}")
        break
    except Exception as e:
        print(f"❌ Failed with encoding {enc}")
        continue

# STEP 3: [Anonymous] reference 제거 함수 정의
def remove_anonymous_refs(ref_string):
    if pd.isna(ref_string):
        return ref_string, 0
    refs = ref_string.split(';')
    filtered_refs = [ref.strip() for ref in refs if not ref.strip().startswith('[Anonymous]')]
    removed_count = len(refs) - len(filtered_refs)
    return '; '.join(filtered_refs) if filtered_refs else None, removed_count

# STEP 4: 적용 및 제거된 개수 카운트
removed_total = 0
cleaned_cr_list = []
for ref in df['CR']:
    cleaned, removed = remove_anonymous_refs(ref)
    cleaned_cr_list.append(cleaned)
    removed_total += removed

df['CR'] = cleaned_cr_list

# STEP 5: 결과 저장
output_file = "sample_cleaned_no_anonymous.csv"
df.to_csv(output_file, index=False, encoding='utf-8-sig')

# STEP 6: 결과 출력 및 다운로드
print(f"✅ 총 제거된 '[Anonymous]' reference 개수: {removed_total}")

files.download(output_file)


Saving 3678_18May.csv to 3678_18May.csv
❌ Failed with encoding utf-8
✅ Encoding detected: cp949
✅ 총 제거된 '[Anonymous]' reference 개수: 7208


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## CSV를 WoS txt 형태로 변환

In [ ]:
# Step 1: 파일 업로드
from google.colab import files
import pandas as pd

uploaded = files.upload()  # 사용자로부터 CSV 업로드 받기

# 업로드된 파일 이름 얻기
filename = next(iter(uploaded))

# Step 2: 인코딩 자동 탐지하여 읽기
encodings_to_try = ['utf-8', 'cp949', 'ISO-8859-1']
for enc in encodings_to_try:
    try:
        df = pd.read_csv(filename, encoding=enc)
        print(f"✅ Encoding detected: {enc}")
        break
    except Exception as e:
        print(f"❌ Failed with encoding: {enc}")
        continue

# Step 3: 변환 함수 정의
def row_to_wos_text(row):
    fields_order = [
        'PT', 'AU', 'AF', 'TI', 'SO', 'LA', 'DT', 'DE', 'ID', 'AB',
        'C1', 'C3', 'RP', 'EM', 'RI', 'OI', 'FU', 'FX', 'CR', 'NR',
        'TC', 'Z9', 'U1', 'U2', 'PU', 'PI', 'PA', 'SN', 'EI', 'J9',
        'JI', 'PD', 'PY', 'VL', 'IS', 'BP', 'EP', 'AR', 'DI', 'EA',
        'PG', 'WC', 'WE', 'SC', 'GA', 'UT', 'DA'
    ]
    lines = []
    for field in fields_order:
        if pd.notna(row.get(field)):
            val = str(row[field]).replace('; ', '\n   ') if field in ['AU', 'AF', 'ID', 'DE'] else str(row[field])
            lines.append(f"{field} {val}")
    lines.append("ER")
    return '\n'.join(lines)

# Step 4: 변환 수행
wos_text = '\n\n'.join(df.apply(row_to_wos_text, axis=1))

# Step 5: 파일로 저장
output_filename = "converted_wos_style.txt"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write(wos_text)

# Step 6: 다운로드 링크 생성
files.download(output_filename)


## CSV를 bib로 변환

In [3]:
# STEP 1: 파일 업로드
from google.colab import files
import pandas as pd
from datetime import datetime

uploaded = files.upload()
filename = next(iter(uploaded))

# STEP 2: CSV 파일 불러오기 (인코딩 자동 탐색)
encodings = ['utf-8', 'cp949', 'ISO-8859-1']
for enc in encodings:
    try:
        df = pd.read_csv(filename, encoding=enc)
        print(f"✅ Encoding detected: {enc}")
        break
    except Exception as e:
        print(f"❌ Failed with encoding {enc}")
        continue

# STEP 3: 한 행을 BibTeX 형식으로 변환하는 함수 정의
def row_to_bibtex(row):
    bib_id = str(row.get('UT', f"generated{datetime.now().timestamp()}")).replace(" ", "_").replace(":", "_")
    bib_entry = [f"@article{{{bib_id},"]

    field_map = {
        'AU': 'Author',
        'TI': 'Title',
        'SO': 'Journal',
        'PY': 'Year',
        'VL': 'Volume',
        'IS': 'Number',
        'BP': 'PagesStart',
        'EP': 'PagesEnd',
        'AB': 'Abstract',
        'DI': 'DOI'
    }

    for csv_key, bib_key in field_map.items():
        value = row.get(csv_key)
        if pd.notna(value):
            if bib_key == 'Author':
                authors = value.replace(';', ' and')
                bib_entry.append(f"  {bib_key} = {{{authors}}},")
            elif bib_key == 'PagesStart':
                bp = value
                ep = row.get('EP')
                pages = f"{bp}-{ep}" if pd.notna(ep) else str(bp)
                bib_entry.append(f"  Pages = {{{pages}}},")
            else:
                bib_entry.append(f"  {bib_key} = {{{value}}},")

    bib_entry.append("}")
    return "\n".join(bib_entry)

# STEP 4: 모든 행을 BibTeX entry로 변환
bibtex_entries = df.apply(row_to_bibtex, axis=1)
bibtex_text = '\n\n'.join(bibtex_entries)

# STEP 5: .bib 파일로 저장
output_file = "converted_from_csv.bib"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(bibtex_text)

# STEP 6: 다운로드 링크 제공
files.download(output_file)


Saving 3678_May15_no저자불명.csv to 3678_May15_no저자불명 (1).csv
✅ Encoding detected: utf-8


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#다른 방법

##텍스트파일 합치기

In [ ]:
from google.colab import files
import io

# 🔹 파일 여러 개 업로드
uploaded = files.upload()

# 🔹 파일 내용을 병합할 리스트
merged_lines = []

# 🔹 업로드된 모든 파일을 순서대로 병합
for filename in uploaded.keys():
    print(f"📄 처리 중: {filename}")
    with open(filename, "r", encoding="utf-8") as f:
        merged_lines.extend(f.readlines())
        merged_lines.append("\n")  # 파일 간 구분을 위한 빈 줄 추가 (선택 사항)

# 🔹 병합된 파일 저장
merged_filename = "merged_output.txt"
with open(merged_filename, "w", encoding="utf-8") as f:
    f.writelines(merged_lines)

print(f"✅ 병합 완료: {merged_filename}")

# 🔹 병합된 파일 다운로드
files.download(merged_filename)


##DT==Article만 남기기

In [ ]:
# 1. 파일 업로드 위젯
from google.colab import files
uploaded = files.upload()

# 2. 업로드된 파일 이름 가져오기
uploaded_filename = next(iter(uploaded))
output_filename = "1_500_article_only.txt"

# 3. DT가 "Article"인 논문만 남기고 나머지 제거
total_records = 0
removed_records = 0
filtered_lines = []
inside_record = False
keep_record = False
current_record = []

with open(uploaded_filename, "r", encoding="utf-8") as f:
    for line in f:
        stripped_line = line.strip()
        if stripped_line.startswith("PT "):
            total_records += 1
            inside_record = True
            keep_record = False
            current_record = [line]
        elif stripped_line.startswith("ER") and inside_record:
            current_record.append(line)
            inside_record = False
            if keep_record:
                filtered_lines.extend(current_record)
            else:
                removed_records += 1
        elif inside_record:
            current_record.append(line)
            if stripped_line.startswith("DT ") and stripped_line[3:].strip() == "Article":
                keep_record = True
        else:
            filtered_lines.append(line)  # 파일 맨 앞이나 끝부분에 있는 부가 정보

# 4. 결과 파일 저장
with open(output_filename, "w", encoding="utf-8") as f:
    f.writelines(filtered_lines)

# 5. 결과 출력
print(f"총 논문 수: {total_records}")
print(f"삭제된 논문 수 (DT != 'Article'): {removed_records}")
print(f"남은 논문 수: {total_records - removed_records}")
print(f"파일 저장 완료: {output_filename}")

# 6. 다운로드 링크 생성
files.download(output_filename)


##참고문헌 중 저자불분명삭제

In [ ]:
# 파일 업로드 위젯 (Colab에서 파일 업로드)
from google.colab import files
uploaded = files.upload()

# 파일 이름 (사용자가 업로드한 파일 이름 자동 추출)
import io

uploaded_filename = next(iter(uploaded))
file_path = uploaded_filename
output_path = "1_500_no_anonymous_preserve_cr.txt"

# [Anonymous] 라인 삭제 + CR 라벨 유지
removed_count = 0
cleaned_lines = []
inside_cr_section = False

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        stripped_line = line.strip()
        if stripped_line.startswith("CR"):
            inside_cr_section = True
            if "[Anonymous]" in stripped_line:
                removed_count += 1
                cleaned_lines.append("CR\n")  # CR은 유지
                continue
        elif inside_cr_section and stripped_line == "":
            inside_cr_section = False
        elif inside_cr_section:
            if "[Anonymous]" in stripped_line:
                removed_count += 1
                continue
        cleaned_lines.append(line)

# 새 파일로 저장
with open(output_path, "w", encoding="utf-8") as f:
    f.writelines(cleaned_lines)

# 결과 출력
print(f"[Anonymous] 라인이 {removed_count}개 삭제되었습니다.")
print(f"정리된 파일이 저장되었습니다: {output_path}")

# 결과저장
files.download("1_500_no_anonymous_preserve_cr.txt")


## 2025년 논문삭제

In [ ]:
# 1. 파일 업로드
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 2. 파일 읽기
with open(filename, 'r', encoding='utf-8') as f:
    data = f.read()

# 3. 논문 블록을 PT ~ ER 단위로 분리
papers = data.split('\nER\n')
papers = [paper.strip() for paper in papers if paper.strip()]

# 4. 'PY 2025' 포함된 논문을 삭제 (PT ~ ER 전체 제거)
filtered_papers = []
deleted_count = 0

for paper in papers:
    if 'PY 2025' in paper:
        deleted_count += 1
    else:
        filtered_papers.append(paper + '\nER')

# 5. 결과 파일로 저장
output_filename = 'cleaned_full_removal_2025.txt'
with open(output_filename, 'w', encoding='utf-8') as f:
    f.write('\n'.join(filtered_papers))

# 6. 결과 출력
print(f'총 논문 수: {len(papers)}')
print(f'"PY 2025" 논문 삭제 수: {deleted_count}')
print(f'남은 논문 수: {len(filtered_papers)}')

# 7. 결과 파일 다운로드
files.download(output_filename)
